<a href="https://colab.research.google.com/github/GiridharIITM/alumni_pathfinder/blob/main/extract_alumni_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import pandas as pd
import os, sys
import bs4 as bs
import urllib.request

alumni_cols = ['id','name',
               'education_start','education_end','education_inst','education_degree','education_branch','education_result',
               'position_start','position_end','position_employer','position_title','position_pay', 'position_workhours',
               'urls_list', 'linkedIn_url'
               ]

BASE_DIR = os.curdir
FILE_DIR = "textfiles"
os.makedirs(os.path.join(BASE_DIR,FILE_DIR), exist_ok=True)

class alumni_extractor:

  def __init__(self, base_url, alumni_url):
    self.base_url = base_url
    self.alumni_url = alumni_url
    self.alumni_history = pd.DataFrame(columns = alumni_cols)
    
  def build_soup(self, url):
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source,'lxml')
    return soup

  # This function actually parses the website, and stores information. 
  # Does a bare minimum population of the alumni_history dataframe.
  # Will be different for different institutes
  def build_data(self):
    pass

  # This function is for NLP functionality
  def enrich_data(self):    
    pass

class iitb_extractor(alumni_extractor):
  def build_data(self):
    next_page_url = self.alumni_url
    max_new_pages = 3
    num_new_pages = 0
    hash_list = []
    name_list = []
    file_list = []
    urls_list = []
    edst_list = []
    ednd_list = []
    eddg_list = []
    edin_list = []
    edbr_list = []
    while(not next_page_url is None):
      soup = self.build_soup(next_page_url)
      for bsname in soup.find_all('td', class_='views-field views-field-title'):
        name = bsname.text.strip()
        id = hash(name) % ((sys.maxsize + 1) * 2)
        url = self.base_url + bsname.a.get('href')
        hash_list.append(id)
        name_list.append(name)
        urls_list.append([url])
        file_list.append(self.build_file(str(id), url))
      for bsqual in soup.find_all('td', class_='views-field views-field-field-iitb-passingout'):
        qual = bsqual.text.strip()
        degree, yr_of_completion, branch = qual.split(', ')
        ednd_list.append([yr_of_completion])
        eddg_list.append([degree])
        edin_list.append(['IIT Bombay'])
        edbr_list.append([branch])
      next_page_url = soup.find('a', title='Go to next page')
      num_new_pages += 1
      if(num_new_pages < max_new_pages and not next_page_url is None):
        next_page_url = self.base_url + next_page_url.get('href')
      else:
        next_page_url = None
    self.alumni_history['id'] = hash_list
    self.alumni_history['name'] = name_list
    self.alumni_history['file_list'] = file_list
    self.alumni_history['urls_list'] = urls_list
    self.alumni_history['education_end'] = ednd_list
    self.alumni_history['education_degree'] = eddg_list
    self.alumni_history['education_inst'] = edin_list
    self.alumni_history['education_branch'] = edbr_list

  def build_file(self, name, url):
    soup = self.build_soup(url)
    fname = os.path.join(BASE_DIR, FILE_DIR, name+'.txt')
    textlines = []
    for section in soup.find_all('div', class_='field-items'):
      textlines.append(section.text.strip() + "\n")
    with open(fname, mode='w') as f:
      f.writelines(textlines)
    return(fname)


In [70]:
iitb = iitb_extractor('https://www.alumni.iitb.ac.in/','https://www.alumni.iitb.ac.in/en/award-list/distinguished-alumnus')
iitb.build_data()
